In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import figure
import random as rd
import copy

In [74]:
class Decreasing_Cascade():
    def __init__(self):
        self.g = nx.DiGraph()
        self.num_nodes = 0
        self.node_label = []
        self.label2id = {}
        self.max_out_degree = 0
        self.probability = None

    def fit(self, g):
        # fit graph with probability
        self.g = g
        self.num_nodes = g.number_of_nodes()
        self.node_label = [i for i in g.nodes()]
        self.label2id = {self.node_label[i]: i for i in range(self.num_nodes)}
        self.max_out_degree = max(j for _, j in g.out_degree(weight='None'))
        out_degree = g.out_degree(weight='None')
        self.probability = np.zeros((self.num_nodes, self.num_nodes), dtype=float)
        for e in g.edges():
            if(out_degree[e[0]] >= 10):
                self.probability[self.label2id[e[0]], self.label2id[e[1]]] = 1 / int(np.log(out_degree[e[0]]))
            else:
                self.probability[self.label2id[e[0]], self.label2id[e[1]]] = 1
        return g
        
    # diffusion to all possible nodes
    def diffusion_all(self, seed_nodes):
        if(seed_nodes == []):
            return [], []
        activated_nodes = [self.label2id[name] for name in seed_nodes]
        old_activated_nodes = copy.deepcopy(activated_nodes)
        activate_nums = [len(activated_nodes)]
        inform_times = np.zeros(self.num_nodes)
        while(True):
            new_activated_nodes = []
            new_inform_times = np.zeros(self.num_nodes)
            for node in old_activated_nodes:
                predecessors = self.probability[:, node].nonzero()[0]
                if(len(predecessors) == 0):
                    continue
                for predecessor in predecessors:
                    if(predecessor in activated_nodes):
                        continue
                    new_inform_times[predecessor] += 1
                    if (self.probability[predecessor][node] >= (rd.random() + inform_times[predecessor] / self.max_out_degree)):
                        new_activated_nodes.append(predecessor)
                activated_nodes.extend(new_activated_nodes)
            if len(new_activated_nodes) == 0:
                break
            old_activated_nodes = new_activated_nodes
            activate_nums.append(len(new_activated_nodes))
            inform_times = inform_times + new_inform_times
        return activated_nodes, activate_nums

    # diffusion to max step
    def diffusion_step(self, seed_nodes, max_step=1):
        if(seed_nodes == []):
            return [], []
        activated_nodes = [self.label2id[name] for name in seed_nodes]
        old_activated_nodes = copy.deepcopy(activated_nodes)
        activate_nums = [len(activated_nodes)]
        inform_times = np.zeros(self.num_nodes)
        for step in range(max_step):
            new_activated_nodes = []
            new_inform_times = np.zeros(self.num_nodes)
            for node in old_activated_nodes:
                predecessors = self.probability[:, node].nonzero()[0]
                if(len(predecessors) == 0):
                    continue
                for predecessor in predecessors:
                    if(predecessor in activated_nodes):
                        continue
                    new_inform_times[predecessor] += 1
                    if (self.probability[predecessor][node] >= (rd.random() + inform_times[predecessor] / self.max_out_degree)):
                        new_activated_nodes.append(predecessor)
                activated_nodes.extend(new_activated_nodes)
            if len(new_activated_nodes) == 0:
                break
            old_activated_nodes = new_activated_nodes
            activate_nums.append(len(new_activated_nodes))
            inform_times = inform_times + new_inform_times
        return activated_nodes, activate_nums

In [3]:
# read from gml
G = nx.read_gml('PB2020.gml')

In [76]:
model = Decreasing_Cascade()
model.fit(G)

In [111]:
sample = rd.sample(list(G.nodes), 10)
activated_nodes, activate_nums = model.diffusion_all(sample)
print(activate_nums)

[10]
